In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
!pip install transformers
import transformers
from transformers import (
    AdamW,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoModel, BertTokenizerFast
from transformers import AutoTokenizer, DataCollatorWithPadding
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# specify GPU
device = torch.device("cuda")

environ({'TERM_PROGRAM': 'Apple_Terminal', 'SHELL': '/bin/zsh', 'TERM': 'xterm-color', 'TMPDIR': '/var/folders/wj/wzcmzk2n2232h1jj_2tfgl700000gn/T/', 'CONDA_SHLVL': '1', 'CONDA_PROMPT_MODIFIER': '(base) ', 'TERM_PROGRAM_VERSION': '444', 'TERM_SESSION_ID': '3AF89B3F-A6F1-40E8-A15B-33FC1A27A66A', 'LC_ALL': 'en_US.UTF-8', 'USER': 'sukaraca', 'CONDA_EXE': '/Users/sukaraca/opt/anaconda3/bin/conda', 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.34nraYaV8T/Listeners', '_CE_CONDA': '', 'CONDA_ROOT': '/Users/sukaraca/opt/anaconda3', 'PATH': '/Users/sukaraca/opt/anaconda3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/Apple/usr/bin', '__CFBundleIdentifier': 'com.apple.Terminal', 'CONDA_PREFIX': '/Users/sukaraca/opt/anaconda3', 'PWD': '/Users/sukaraca', 'LANG': 'en_US.UTF-8', 'EVENT_NOKQUEUE': '1', 'XPC_FLAGS': '0x0', 'XPC_SERVICE_NAME': '0', '_CE_M': '', 'HOME': '/Users/sukaraca', 'SHLVL': '4', 'LOGNAME': 'sukaraca', 'CONDA_PYTHON_EXE': '/Users/sukaraca/opt/anaconda3/bin/python', '

In [2]:
df_train = pd.read_csv('polusa_data/left_train.csv')
df_test = pd.read_csv('polusa_data/left_test.csv')
df_val = pd.read_csv('polusa_data/left_val.csv')

In [10]:
#load model and tokenizer

bert = AutoModel.from_pretrained('bert-base-uncased', num_labels=2, return_dict=False)

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# tokenize and encode sequences
max_length = 512

tokens_train = tokenizer.__call__(
    df_train['body'].tolist(),
    padding = True,
    truncation = True)
  
tokens_val = tokenizer.__call__(
    df_val['body'].tolist(),
    padding = True,
    truncation = True)
  
tokens_test = tokenizer.__call__(
    df_test['body'].tolist(),
    padding = True,
    truncation = True)

/Users/sukaraca/opt/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
#convert the integer sequences to tensors.
  
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(df_train['label'].tolist())
  
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(df_val['label'].tolist())
  
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(df_test['label'].tolist())

In [ ]:
#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
#THINGS TO BE CAREFUL ABOUT
    #FREEZING ONLY 3 LAYERS
    #1 DENSE LAYER , 1 OUTPUT LAYER

    #defining new layers
class BERT_left_layer(nn.Module):
  
    def __init__(self, bert):
        
        super(BERT_left_layer, self).__init__()
  
        self.bert = bert 
        
      # dropout layer
        self.dropout = nn.Dropout(0.2)
        
      # relu activation function
        self.relu =  nn.ReLU()
  
      # dense layer 1
        self.fc1 = nn.Linear(768,512)
        
      # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,2)
  
      #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)
  
    #define the forward pass
    def forward(self, sent_id, mask):
  
      #pass the inputs to the model  
        _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
        
        x = self.fc1(cls_hs)
  
        x = self.relu(x)
  
        x = self.dropout(x)
  
      # output layer
        x = self.fc2(x)
        
    # apply softmax activation
        x = self.softmax(x)
  
        return x

In [ ]:
model = BERT_left_layer(bert)

#push the model to GPU
model = model.to(device)

optimizer = AdamW(model.parameters(),lr = 1e-5)  # learning rate

In [ ]:
#compute the class weights
class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(df_train['label']), y = df_train['label'])

print("Class Weights:",class_weights)

In [ ]:
# converting list of class weights to a tensor
weights= torch.tensor(class_weights,dtype=torch.float)

# push to GPU
weights = weights.to(device)

# define the loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 3

In [ ]:
#fine-tuning

# function to train the model
def train():
  
    model.train()

    total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
    total_preds=[]
  
  # iterate over batches
    for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
        batch = [r.to(device) for r in batch]
 
        sent_id, mask, labels = batch

    # clear previously calculated gradients 
        model.zero_grad()        

    # get model predictions for the current batch
        preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)

    # add on to the total loss
        total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
        loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
        optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()

    # append the model predictions
        total_preds.append(preds)

  # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
    return avg_loss, total_preds

In [ ]:
#Evaluation

# function for evaluating the model
def evaluate():
  
    print("\nEvaluating...")
  
  # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
    total_preds = []

  # iterate over batches
    for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
      # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

    # deactivate autograd
        with torch.no_grad():
      
      # model predictions
            preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
            loss = cross_entropy(preds,labels)

            total_loss = total_loss + loss.item()

            preds = preds.detach().cpu().numpy()

            total_preds.append(preds)

  # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [ ]:
#fine-tuning

# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

In [ ]:
#predictions

#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

# get predictions for test data
with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()

In [ ]:
#model's performance

preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

In [ ]:
# confusion matrix
pd.crosstab(test_y, preds)